In [1]:
print('e')

e


In [1]:
import re
import PyPDF2
import io
import google.generativeai as genaiI
from langchain_google_genai import GoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from concurrent.futures import ThreadPoolExecutor
import os
import tempfile
import json
from langchain_community.document_loaders import PyPDFLoader
# from langchain_google_genai import GoogleGenerativeAIEmbeddings


c:\Users\UTKARSH\.conda\envs\aiagent\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pdf_path = "C:\AI-Recruitment-agent\datascienceresume32 - Google Docs.pdf"

In [3]:
loder = PyPDFLoader(pdf_path)


In [4]:
documents = loder.load()

# print(documents[0].page_content)


In [5]:
from dotenv import load_dotenv
load_dotenv()
GOOGLE_API_KEYY = os.getenv("GEMINI_API")
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=GOOGLE_API_KEYY)


In [46]:
result = llm.invoke('hii')
print(result.content)

Hi there! How can I help you today?



In [47]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GOOGLE_API_KEYY)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,      # Maximum size of each chunk
    chunk_overlap=200,    # Overlap between chunks to maintain context
    length_function=len  # Function to measure chunk length
    
)
chunks = text_splitter.split_documents(documents)
texts = [doc.page_content for doc in chunks]  # Extract text from Document objects
vectorstore = FAISS.from_texts(texts, embeddings)



In [48]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context:
{context}

Question: {input} 
""")
     



In [49]:
retriever = vectorstore.as_retriever()


In [50]:
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm, prompt)

In [ ]:
from langchain.chains import create_retrieval_chain
ra_chain = create_retrieval_chain(retriever, document_chain)


In [52]:
response = ra_chain.invoke({"input": 'What is my fathers name?'})

In [53]:
print(response['answer'])

I'm sorry, but this document does not contain information about the father's name.



In [15]:
print(response['context'][0].page_content)

●  Data Science Certification  -  Coursera  November 2023 
 ●  Earned Badge of Python and SQL  –  HackerRank  January 2024 
 ●  Five Star Python Coder -  HackerRank  May 2024 
 ADDITIONAL INFORMA TION 
 ●  Location :  Delhi 
 ●  Availability :  Immediate Joiner


In [16]:
skills_to_analyze = [
    "Python",
    "Java",
    "React",
    "PostgreSQL",
    "Cloud Computing (AWS/Azure/GCP)",
    "Leadership",
    "Project Management",
    "Machine Learning" # Example of a skill that might not be explicitly mentioned
]


In [21]:
# skill_scores = []
# for skill in skills_to_analyze:
#     query = f"On a scale of 0-10, how clearly does the candidate mention proficiency in {skill}? Provide a numeric rating first, followed by resoning"
#     response = retrieval_chain.invoke({"input": query})
#     match = re.search(r'(\d+)', response['answer'])
#     score = int(match.group(1) if match else 0)
#     reasoning_parts = response['answer'].split('\n', 1)
#     if len(reasoning_parts) > 1:
            
#             reasoning = reasoning_parts[1].strip()
#     else:
#             reasoning = response['answer'].strip()
#     # reasoning = response['answer'].split('\n', 1)[1].strip() if '.' in response['answer'] and len(response['answer'].split('.')) > 1 else ""
#     final_score = min(score, 10)
#     skill_scores.append(final_score)
#     print(f"  Score for {skill}: {final_score}/10")
#     print(f"  Reasoning: {reasoning}")
# # Overall score calculation
    
# # overall_score = sum(skill_scores)
# # n = len(skill_scores)
# # print(f"Overall Score: {overall_score}/{n * 10}")
# # find overll score
# print(skill_scores)





In [58]:
# final code
skill_scores = []
for skill in skills_to_analyze:
    query = f"On a scale of 0-10, how clearly does the candidate mention proficiency in {skill}? Provide a numeric rating first, followed by resoning"
    response = ra_chain.invoke({"input": query})
    match = re.search(r'(\d+)', response['answer'])
    score = int(match.group(1) if match else 0)
    reasoning_parts = response['answer']
    reasining_lines = [line.strip() for line in reasoning_parts.split('\n') if line.strip()]
    
    raw_reasoning = ' '.join(reasining_lines)
   
    final_resoning = re.sub(r'^\d+\s*[-.:]?\s*', '', raw_reasoning)
    # if len(reasoning_parts) > 1:
            
    #         reasoning = reasoning_parts[1].strip()
    # else:
    #         reasoning = response['answer'].strip()
    # reasoning = response['answer'].split('\n', 1)[1].strip() if '.' in response['answer'] and len(response['answer'].split('.')) > 1 else ""
    final_score = min(score, 10)
    skill_scores.append(final_score)
    print(f"  Score for {skill}: {final_score}/10")
    print(f"  Reasoning: {final_resoning}")
# Overall score calculation
    
overall_score = sum(skill_scores)
n = len(skill_scores)
print(f"Overall Score: {overall_score}/{n * 10}")



  Score for Python: 10/10
  Reasoning: The candidate explicitly lists Python under "Programming Languages," earned a "Five Star Python Coder" badge, and has multiple projects and experiences demonstrating Python skills.
  Score for Java: 0/10
  Reasoning: The document does not mention Java at all.
  Score for React: 0/10
  Reasoning: React is not mentioned in the provided context.
  Score for PostgreSQL: 0/10
  Reasoning: The candidate does not mention PostgreSQL.
  Score for Cloud Computing (AWS/Azure/GCP): 0/10
  Reasoning: The candidate does not mention any Cloud Computing skills (AWS/Azure/GCP) in the provided context.
  Score for Leadership: 0/10
  Reasoning: The candidate does not mention leadership skills or experience in the provided context.
  Score for Project Management: 0/10
  Reasoning: The candidate does not mention proficiency in Project Management.
  Score for Machine Learning: 10/10
  Reasoning: /10. The candidate mentions Machine Learning under "Technical Skills", and

In [ ]:
weaknesses  = []
improvement_suggestions = {}

analysis_result = {
    "missing_skill": ["Mlops", "Devops", 'GenerativeAI']
}

for skill in analysis_result.get('missing_skill', []):
    prompt = f"""
    Analyze why the resume is weak in demonstrating proficiency in "{skill}".

    For your analysis, consider:
    1. What's missing from the resume regarding this skill?
    2. How could it be improved with specific examples?
    3. What specific action items would make this skill stand out?

    Provide your response in this JSON format:
    {{
        "weakness": "A concise description of what's missing or problematic (1-2 sentences)",
        "improvement_suggestions": [
            "Specific suggestion 1",
            "Specific suggestion 2",
            "Specific suggestion 3"
        ],
        "example_addition": "A specific bullet point that could be added to showcase this skill"
    }}

    Return only valid JSON, no other text.
    """
    
    response = ra_chain.invoke({"input": prompt})
    
    # ✅ Extract string answer
    raw_json = response['answer']

    # ✅ Remove backticks and 'json' label
    cleaned_json = re.sub(r'^```json|```$', '', raw_json.strip(), flags=re.MULTILINE).strip()

    try:
        # ✅ Parse string into dictionary
        weakness_data = json.loads(cleaned_json)

        # ✅ Store in desired format
        weakness_detail = {
            "skill": skill,
            "detail": weakness_data.get("weakness", "No specific details provided."),
            "suggestions": weakness_data.get("improvement_suggestions", []),
            "example": weakness_data.get("example_addition", "")
        }

        weaknesses.append(weakness_detail)
        improvement_suggestions[skill] = {
                    "suggestions": weakness_data.get("improvement_suggestions", []),
                    "example": weakness_data.get("example_addition", "")}

    except json.JSONDecodeError as e:
        print(f"JSON parsing failed for skill {skill}: {e}")
        weaknesses.append({
            "skill": skill,
            "detail": raw_json[:200],  # fallback: first 200 characters
            
            "example": ""
        })

resume_weakness = weaknesses

In [65]:
for item in resume_weakness:
    print(f"\n🔧 Skill: {item['skill']}")
    print(f"❌ Weakness: {item['detail']}")
    print(f"📌 Example: {item['example']}")
    print("✅ Suggestions:")
    for s in item['suggestions']:
        print(f"  - {s}")


🔧 Skill: Mlops
❌ Weakness: The resume is weak in demonstrating proficiency in MLOps because it lacks any explicit mention of MLOps practices, tools, or experience with deploying and managing machine learning models in a production environment. While the resume highlights model building and accuracy, it doesn't showcase understanding of the complete ML lifecycle.
📌 Example: Implemented CI/CD pipeline using GitHub Actions and Docker to automate model deployment for the Gemstone Price Predictor, reducing deployment time by 50%.
✅ Suggestions:
  - Include experience with model deployment tools like Docker, Kubernetes, AWS SageMaker, or similar.
  - Mention any involvement in model monitoring, retraining, or version control processes.
  - Specify if any CI/CD pipelines were used for model deployment.

🔧 Skill: Devops
❌ Weakness: The resume is weak in demonstrating proficiency in DevOps because it lacks any explicit mention or demonstration of skills, tools, or experiences related to DevOps

In [78]:
for skill, suggestion_data in improvement_suggestions.items():
    print(f"\n🔧 Skill: {skill}")
    
    print("✅ Suggestions:")
    for suggestion in suggestion_data.get("suggestions", []):
        print(f"  - {suggestion}")
    
    print(f"📌 Example addition: {suggestion_data.get('example', 'No example provided.')}")



🔧 Skill: Mlops
✅ Suggestions:
  - Mention experience with containerization tools like Docker or Kubernetes.
  - Include experience with CI/CD pipelines for model deployment.
  - Add information about model monitoring and performance tracking tools.
📌 Example addition: Implemented CI/CD pipelines using Jenkins to automate model retraining and deployment, reducing deployment time by 50% and ensuring consistent performance monitoring.

🔧 Skill: Devops
✅ Suggestions:
  - Include any experience with cloud platforms (AWS, Azure, GCP) that might involve infrastructure management or deployment.
  - Mention any experience with containerization technologies like Docker or orchestration tools like Kubernetes.
  - If any projects involved automation of deployment or testing, highlight those aspects.
📌 Example addition: Automated deployment of the Gemstone Price Predictor using Docker and GitHub Actions, resulting in a 20% reduction in deployment time.

🔧 Skill: GenerativeAI
✅ Suggestions:
  - Inc

In [ ]:
# def extract_skills_jd(jd_text):
"Extract skills from job description"
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=GOOGLE_API_KEYY)
jd_text = """
Job Title: Data Scientist
Location: Remote / Bangalore, India
Job Type: Full-time

About the Role:
We are seeking a Data Scientist who is passionate about using data to solve real-world problems and drive strategic decisions. You'll work closely with business, product, and engineering teams to uncover insights, build predictive models, and design experiments that impact millions of users.

Key Responsibilities:
- Analyze structured and unstructured data from multiple sources to extract meaningful insights.
- Build machine learning models for prediction, classification, segmentation, and recommendation.
- Design A/B tests and evaluate the performance of models and features.
- Communicate findings clearly to stakeholders using dashboards, reports, and visualizations.
- Collaborate with data engineers and software developers to deploy scalable data solutions.
- Monitor model performance and perform periodic model retraining.

Required Skills & Qualifications:
- Bachelor's or Master's degree in Computer Science, Statistics, Mathematics, or related field.
- 2+ years of experience in data science or a similar role.
- Strong knowledge of Python, SQL, and machine learning libraries like scikit-learn, TensorFlow, or PyTorch.
- Hands-on experience with data analysis and visualization tools such as Pandas, Matplotlib, Seaborn, or Tableau.
- Understanding of statistical modeling, hypothesis testing, and experimental design.
- Experience working with large datasets and cloud platforms (AWS, GCP, or Azure).
"""

prompt = f"""
            Extract a comprehensive list of technical skills, technologies, and competencies required from this job description. 
            Format the output as a Python list of strings. Only include the list, nothing else.
            
            Job Description:
            {jd_text}
            """

response = llm.invoke(prompt)



In [ ]:
skill_text = response.content
cleaned = re.sub(r"```python|```", "",skill_text).strip()

jd_skills = eval(cleaned)
print(jd_skills)


In [6]:
resume_text = """
Utkarsh Raj
Email: utkarsh@example.com | Phone: +91-9876543210 | Location: Rohtak, Haryana, India
LinkedIn: www.linkedin.com/in/utkarsh9911 | GitHub: github.com/utkarshraj99

Objective:
Aspiring Data Scientist with hands-on experience in building machine learning models, performing data analysis, and implementing MLOps tools. Seeking to leverage my skills to solve real-world business problems and deliver data-driven solutions.

Education:
Bachelor of Science in Computer Science (2021 – 2025)
Maharshi Dayanand University, Rohtak, Haryana

Skills:
- Programming Languages: Python, SQL
- Libraries & Tools: Pandas, NumPy, Scikit-learn, TensorFlow, Matplotlib, Seaborn, NLTK, SpaCy
- MLOps Tools: DVC, MLflow, Apache Airflow, Docker, Git
- Web Development: Flask, Streamlit, FastAPI
- Cloud Platforms: Google Cloud Storage
- Version Control: Git, GitHub
- Data Engineering: YAML, JSON, CSV handling, Logging & Error Handling

Projects:

1. Stock Price Prediction using LSTM
   - Built and trained an LSTM model using historical Apple stock data via yfinance
   - Performed data preprocessing, scaling, and model evaluation
   - Technologies: TensorFlow, Pandas, Matplotlib

2. Resume Analyzer & Interview Bot
   - Built a Streamlit app that evaluates resumes, gives ATS scores, and suggests improvements
   - Includes features to generate interview questions and analyze job descriptions
   - Integrated RAG-based Q&A with LLMs

3. Customer Lifetime Value Prediction (CLTV)
   - Performed RFM segmentation and applied BetaGeoFitter and GammaGamma models
   - Developed marketing strategy recommendations for each customer cluster

4. Hate Speech Detection (NLP)
   - Created a dataset pipeline with MongoDB, Zip extraction, and preprocessing
   - Built models to classify hate speech in tweets using logistic regression and NLP techniques

Certifications:
- AI For Everyone – Coursera
- Data Science & Machine Learning – Kaggle Learn

Achievements:
- Completed over 10 real-world data science projects
- Contributed to open-source ML templates using DVC and MLflow

Languages:
- English (Intermediate), Hindi (Native)

Declaration:
I hereby declare that the above information is true to the best of my knowledge.
"""


In [8]:
def create_vector_store(text):
    """Create a simpler vector store for skill analysis"""
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GOOGLE_API_KEYY)
    text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,      # Maximum size of each chunk
    chunk_overlap=200,    # Overlap between chunks to maintain context
    length_function=len ) # Function to measure chunk length
    
    chunks = text_splitter.split_text(text)
    vectorstore = FAISS.from_texts(chunks, embeddings)
    return vectorstore

def analyze_skills(ra_chain, skill):
    
    """Analyze a single skill using the RAG chain and return skill, score, and cleaned reasoning."""
        
        # Ask the chain for evaluation of the skill
    query = f"On a scale of 0-10, how clearly does the candidate mention proficiency in {skill}? Provide a numeric rating first, followed by reasoning"
    response = ra_chain.invoke({"input": query})

        # Extract numeric score from the response
    match = re.search(r'(\d+)', response['answer'])
    score = int(match.group(1)) if match else 0
        # final_score = min(score, 10)

        # Clean up the reasoning part
    reasoning_raw = response['answer']
    reasoning_lines = [line.strip() for line in reasoning_raw.split('\n') if line.strip()]
    raw_reasoning = " ".join(reasoning_lines)

        # Remove leading score and optional symbols
    reasoning = re.sub(r'^\d+\s*[-.:]?\s*', '', raw_reasoning)

    return skill, min(score, 10), reasoning

In [70]:
response = qa_chain.invoke({"query": "What skills does the resume show in MLOps?"})

In [35]:
print(response['result'])

Based on the resume, Utkarsh Raj has experience with the following MLOps tools:

*   DVC
*   MLflow
*   Apache Airflow
*   Docker
*   Git


In [9]:
skills = [
    "Python",
    "Java",
    "React",
    "PostgreSQL",
    "Cloud Computing (AWS/Azure/GCP)",
    "Leadership",
    "Project Management",
    "Machine Learning" # Example of a skill that might not be explicitly mentioned
]

In [11]:
skill_scores = {}
skill_reasoning = {}
missing_skills = []
total_score = 0

from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context:
{context}

Question: {input} 
""")


vectorstore = create_vector_store(resume_text)
retriever = vectorstore.as_retriever()

     
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm, prompt)
from langchain.chains import create_retrieval_chain
ra_chain = create_retrieval_chain(retriever, document_chain)

skills_analayze = [
    "Python",
    "Java",
    "React",
    "PostgreSQL",
    "Cloud Computing (AWS/Azure/GCP)",
    "Leadership",
    "Project Management",
    "Machine Learning" # Example of a skill that might not be explicitly mentioned
]


with ThreadPoolExecutor(max_workers=5) as executor:
        results = list(executor.map(lambda skill: analyze_skills(ra_chain, skill), skills))


for skill, score, reasoning in results:

    skill_scores[skill] = score
    skill_reasoning[skill] = reasoning
    total_score = total_score + score
    if score <=5:
          missing_skills.append(skill)
    
    # print(f"Raw response for {skill}: {response['answer']}")
    
overall_score = int((total_score / (10 *len(skills)))*100)

selected = overall_score >= 50

reasoning = "Candidate evaluated based on explicit resume content using semantic similarity and clear numeric scoring."
strengths = [skill for skill, score in skill_scores.items() if score >= 7]
improvement_areas = missing_skills if not selected else []

print(overall_score)
print(skill_scores)
print(skill_reasoning)
print(selected)
print(reasoning)
print(missing_skills)
print(strengths)
print(improvement_areas)



25
{'Python': 9, 'Java': 0, 'React': 0, 'PostgreSQL': 0, 'Cloud Computing (AWS/Azure/GCP)': 2, 'Leadership': 0, 'Project Management': 0, 'Machine Learning': 9}
{'Python': 'The candidate explicitly lists Python as a programming language under the "Skills" section. The extensive list of libraries and tools also implies a strong proficiency in Python.', 'Java': 'The document does not mention Java.', 'React': 'The provided text does not mention React at all.', 'PostgreSQL': 'Reasoning: PostgreSQL is not mentioned anywhere in the provided document.', 'Cloud Computing (AWS/Azure/GCP)': 'The candidate only mentions Google Cloud Storage in the skills section. There is no indication of proficiency, or even exposure, to cloud computing beyond storage.', 'Leadership': 'The candidate does not mention proficiency in Leadership.', 'Project Management': "Project management proficiency is not explicitly mentioned. While the candidate lists completed projects, there's no discussion of project managemen

In [13]:
weaknesses  = []
improvement_suggestions = {}

analysis_result = {
    "missing_skill": ["Mlops", "Devops", 'GenerativeAI']
}

for skill in analysis_result.get('missing_skill', []):
    prompt = f"""
    Analyze why the resume is weak in demonstrating proficiency in "{skill}".

    For your analysis, consider:
    1. What's missing from the resume regarding this skill?
    2. How could it be improved with specific examples?
    3. What specific action items would make this skill stand out?

    Resume Content:
    {resume_text[:30000]}

    Provide your response in this JSON format:
    {{
        "weakness": "A concise description of what's missing or problematic (1-2 sentences)",
        "improvement_suggestions": [
            "Specific suggestion 1",
            "Specific suggestion 2",
            "Specific suggestion 3"
        ],
        "example_addition": "A specific bullet point that could be added to showcase this skill"
    }}

    Return only valid JSON, no other text.
    """
    
    response = llm.invoke(prompt)
    
    # ✅ Extract string answer
    raw_json = response.content

    # ✅ Remove backticks and 'json' label
    cleaned_json = re.sub(r'^```json|```$', '', raw_json.strip(), flags=re.MULTILINE).strip()

    try:
        # ✅ Parse string into dictionary
        weakness_data = json.loads(cleaned_json)

        # ✅ Store in desired format
        weakness_detail = {
            "skill": skill,
            "detail": weakness_data.get("weakness", "No specific details provided."),
            "suggestions": weakness_data.get("improvement_suggestions", []),
            "example": weakness_data.get("example_addition", "")
        }

        weaknesses.append(weakness_detail)
        improvement_suggestions[skill] = {
                    "suggestions": weakness_data.get("improvement_suggestions", []),
                    "example": weakness_data.get("example_addition", "")}

    except json.JSONDecodeError as e:
        print(f"JSON parsing failed for skill {skill}: {e}")
        weaknesses.append({
            "skill": skill,
            "detail": raw_json[:200],  # fallback: first 200 characters
            
            "example": ""
        })

resume_weakness = weaknesses

In [14]:
for item in resume_weakness:
    print(f"\n🔧 Skill: {item['skill']}")
    print(f"❌ Weakness: {item['detail']}")
    print(f"📌 Example: {item['example']}")
    print("✅ Suggestions:")
    for s in item['suggestions']:
        print(f"  - {s}")


🔧 Skill: Mlops
❌ Weakness: The resume lists MLOps tools but lacks demonstrable evidence of using them effectively in a complete MLOps workflow. It doesn't showcase practical application beyond simply knowing the tool names.
📌 Example: Implemented a CI/CD pipeline using GitHub Actions, Docker, and MLflow to automate model retraining and deployment, resulting in a 30% reduction in model deployment time.
✅ Suggestions:
  - Quantify the impact of using MLOps tools. Instead of just stating the tools used, highlight how their use improved model deployment frequency, reduced errors, or enhanced model performance.
  - Focus on showcasing specific MLOps workflows implemented, detailing the end-to-end process. For example, explicitly describe the process of model versioning, CI/CD pipeline automation, and monitoring infrastructure used.
  - Replace generic descriptions with quantifiable achievements. For example, instead of 'Contributed to open-source ML templates using DVC and MLflow,' provide

In [15]:
analysis_result = {'strengths':{'python', 'machine learning'}}
extracted_skills = [
    "Python",
    "Java",
    "React",
    "PostgreSQL",
    "Cloud Computing (AWS/Azure/GCP)",
    "Leadership",
    "Project Management",
    "Machine Learning" 
]
num_questions = 3
difficulty = 'Medium'
question_types = ['technical', 'Behavioural']
context = f"""
Resume Content:
{resume_text[:2000]}...
            
Skills to focus on: {', '.join(extracted_skills)}
            
Strengths: {', '.join(analysis_result.get('strengths', []))}
            
Areas for improvement: {', '.join(missing_skills)}
            """
            
prompt = f"""
Generate {num_questions} personalized {difficulty.lower()} level interview questions for this candidate 
based on their resume and skills. Include only the following question types: {', '.join(question_types)}.
            
For each question:
1. Clearly label the question type
2. Make the question specific to their background and skills
3. For coding questions, include a clear problem statement
            
{context}
            
Format the response as a list of tuples with the question type and the question itself.
Each tuple should be in the format: ("Question Type", "Full Question Text")
"""

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=GOOGLE_API_KEYY)
response =  llm.invoke(prompt)



In [59]:
print(response.content)
questions_text = response.content

Here are three interview questions tailored for Utkarsh Raj, based on his resume and skills, in the requested format:

[
    ("Technical", "Based on your project 'Hate Speech Detection (NLP)', describe a situation where you encountered imbalanced classes in the dataset. Explain what techniques you used to address this imbalance and how they affected the performance of your logistic regression model. Provide sample code in python of how you handled class imbalance. "),
    ("Behavioural", "You've worked on multiple data science projects, including the 'Customer Lifetime Value Prediction (CLTV)' project. Describe a time when you had to communicate complex analytical findings and marketing strategy recommendations to stakeholders with limited technical knowledge. How did you tailor your communication style to ensure they understood the key insights and could make informed decisions?"),
    ("Technical", "In your resume, you mentioned using DVC for MLOps in your projects. Imagine you are w

In [57]:
questions_text = response.content
questions = []

# Step 1: Remove any extra explanation text before the list
match = re.search(r"\[\s*\(", questions_text, re.DOTALL)
if match:
    start_index = match.start()
    questions_text = questions_text[start_index:]

# Step 2: Clean up Markdown or Python syntax
questions_text = re.sub(r"```(?:python)?|```", "", questions_text).strip()
print(questions_text)

# Step 3: Safely evaluate the string into a Python list
try:
    questions_list = eval(questions_text)
    questions = []
    for question_type, question in questions_list:
        for requested_type in question_types:
            if requested_type.lower() in question_type.lower():
                questions.append((question_type.strip(), question.strip()))
                break
except Exception as e:
    print(f"Parsing failed: {e}")
    questions = []


[
    ("Technical", "Based on your project 'Hate Speech Detection (NLP)', describe a situation where you encountered imbalanced classes in the dataset. Explain what techniques you used to address this imbalance and how they affected the performance of your logistic regression model. Provide sample code in python of how you handled class imbalance. "),
    ("Behavioural", "You've worked on multiple data science projects, including the 'Customer Lifetime Value Prediction (CLTV)' project. Describe a time when you had to communicate complex analytical findings and marketing strategy recommendations to stakeholders with limited technical knowledge. How did you tailor your communication style to ensure they understood the key insights and could make informed decisions?"),
    ("Technical", "In your resume, you mentioned using DVC for MLOps in your projects. Imagine you are working on a project with a team and are using DVC to manage different versions of your models and datasets. Could you d

In [39]:
try:
    questions_list = eval(questions_text1)
    print(questions_list)
    print(questions_list[0].strip())
except Exception as e:
    print(e)

[('Technical', "Based on your project 'Hate Speech Detection (NLP)', describe a situation where you encountered imbalanced classes in the dataset. Explain what techniques you used to address this imbalance and how they affected the performance of your logistic regression model. Provide sample code in python of how you handled class imbalance. "), ('Behavioural', "You've worked on multiple data science projects, including the 'Customer Lifetime Value Prediction (CLTV)' project. Describe a time when you had to communicate complex analytical findings and marketing strategy recommendations to stakeholders with limited technical knowledge. How did you tailor your communication style to ensure they understood the key insights and could make informed decisions?"), ('Technical', 'In your resume, you mentioned using DVC for MLOps in your projects. Imagine you are working on a project with a team and are using DVC to manage different versions of your models and datasets. Could you describe a sce

In [63]:
abc = [
    ("Technical", "Q1 about hate speech project..."),
    ("Behavioural", "Q2 about communication..."),
    ("Technical", "Q3 about DVC and MLOps...")
]
question_types = ["Technical"]

questions = []

for question_type, question in abc:
    for requested_type in question_types:
        if requested_type.lower() in question_type.lower():
            questions.append((question_type.strip(), question.strip()))
            
print(questions)


[('Technical', 'Q1 about hate speech project...'), ('Technical', 'Q3 about DVC and MLOps...')]
